In [3]:
import os
os.getcwd()

'/Users/fmarinperucci/datascience/learning/uw_python_datascience/case_studies'

In [6]:
import graphlab
products = graphlab.SFrame('amazon_baby.gl/')

#Hola, ahora start assignment

In [7]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [8]:
graphlab.canvas.set_target('ipynb')

In [9]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [12]:
type(products['word_count'][0])

dict

In [15]:
'and' in products['word_count'][0]

True

In [34]:
def myword_count(dict,myword):
    if myword in dict:
        return dict[myword]
    else:
        return 0


In [35]:
myword_count(products['word_count'][0],'and')

5

In [25]:
products['awesome'] = products['word_count'].apply(awesome_count)

In [38]:
products[selected_words[1]] = products['word_count'].apply(lambda x:myword_count(x,selected_words[1]))

### find which selected word was used the most - create new columns using 'apply' - check use of lambda

In [47]:
for sword in selected_words:
    products[sword] = products['word_count'].apply(lambda x:myword_count(x,sword))
    print '%s = %i occurrences' % (sword, sum(products[sword]))


awesome = 2090 occurrences
great = 45206 occurrences
fantastic = 932 occurrences
amazing = 1363 occurrences
love = 42065 occurrences
horrible = 734 occurrences
bad = 3724 occurrences
terrible = 748 occurrences
awful = 383 occurrences
wow = 144 occurrences
hate = 1220 occurrences


### add the semtiment

In [52]:
#ignore all 3* reviews
products = products[products['rating'] != 3]
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

##divide in training and testing samples

In [55]:
train_data,test_data = products.random_split(.8, seed=0)

In [56]:
train_data.column_names()

['name',
 'review',
 'rating',
 'word_count',
 'awesome',
 'great',
 'fantastic',
 'amazing',
 'love',
 'horrible',
 'bad',
 'terrible',
 'awful',
 'wow',
 'hate',
 'sentiment']

In [57]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 1.265854     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 1.438784     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 1.600737     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [67]:
selected_words_model['coefficients'].print_rows(num_rows=15)

+-------------+-------+-------+------------------+
|     name    | index | class |      value       |
+-------------+-------+-------+------------------+
| (intercept) |  None |   1   |  1.36728315229   |
|   awesome   |  None |   1   |  1.05800888878   |
|    great    |  None |   1   |  0.883937894898  |
|  fantastic  |  None |   1   |  0.891303090304  |
|   amazing   |  None |   1   |  0.892802422508  |
|     love    |  None |   1   |  1.39989834302   |
|   horrible  |  None |   1   |  -1.99651800559  |
|     bad     |  None |   1   | -0.985827369929  |
|   terrible  |  None |   1   |  -2.09049998487  |
|    awful    |  None |   1   |  -1.76469955631  |
|     wow     |  None |   1   | -0.0541450123333 |
|     hate    |  None |   1   |  -1.40916406276  |
+-------------+-------+-------+------------------+
[12 rows x 4 columns]



In [74]:
selected_words_model.evaluate(test_data,metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   3e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   4e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   5e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   6e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   7e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   8e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   9e-05   | 1.0 | 1.0 | 27976 | 5328 |
 +-----------+-----+-----+-------+------+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [75]:
selected_words_model.show(view='Evaluation')

In [77]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']
len(diaper_champ_reviews)

298

In [79]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)
diaper_champ_reviews.head()

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'rating': 1,'contacted': 1, 'over': ...",0,1,0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'bags.': 1, ""don't"": 1,'son.': 1, 'of,': 1, ...",0,0,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'instead': 1, 'all': 1,'already': 1, 'love': 3, ...",0,0,0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'and': 2, 'bags.': 1,'garbage': 1, 'wastef ...",0,2,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'and': 3, 'love': 1,'use.': 1, 'is': 2, ' ...",0,2,0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,"{'and': 4, 'genies': 1,'all': 1, 'because': 1, ...",0,0,0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'and': 1, 'old': 1,'extra': 1, 'is': 1, ...",0,0,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3, 'money': 1,'still': 3, 'fine': 1, ...",0,0,0
Baby Trend Diaper Champ,Love the Diaper Champ. Ihad planned to get the ...,4.0,"{'reviews,': 1, 'infant':1, 'bags.': 1, 'just' ...",0,0,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1, 'both': 1,'(required': 1, 'befo ...",0,0,0


In [85]:
diaper_champ_reviews[diaper_champ_reviews['rating']==5.0].print_rows(num_rows=100)

+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | I received my Diaper Champ... |  5.0   |
| Baby Trend Diaper Champ | Love it, love it, love it!... |  5.0   |
| Baby Trend Diaper Champ | Works great - no smells.  ... |  5.0   |
| Baby Trend Diaper Champ | I love this diaper pale an... |  5.0   |
| Baby Trend Diaper Champ | I've worked with kids more... |  5.0   |
| Baby Trend Diaper Champ | This is absolutely, by far... |  5.0   |
| Baby Trend Diaper Champ | I have a two-year-old son ... |  5.0   |
| Baby Trend Diaper Champ | I have a 10 year old daugh... |  5.0   |
| Baby Trend Diaper Champ | I love this diaper pail!  ... |  5.0   |
| Baby Trend Diaper Champ | My son is now 18 months ol... |  5.0   |
| Baby Trend Diaper Champ | I have been using this dia... |  5.0   |
| Baby Trend Diaper Champ | I have

In [95]:
diaper_champ_reviews[diaper_champ_reviews['rating']==5.0][91]

{'amazing': 0,
 'awesome': 0,
 'awful': 0,
 'bad': 0,
 'fantastic': 0,
 'great': 0,
 'hate': 0,
 'horrible': 0,
 'love': 0,
 'name': 'Baby Trend Diaper Champ',
 'predicted_sentiment': 0.7969408512906712,
 'rating': 5.0,
 'review': 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor e